In [1]:
#IMPORT LIBRARIES
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import random
import re
from bs4 import BeautifulSoup
import requests
import lxml
from lxml import html
import os
import pandas as pd

## 2. Search Engine

### 2.0 Preprocessing

#### 2.0.0) Preprocessing the text

Firstly, we need to read the contents of the csv file that we obtained in Q1.

In [2]:
dataset = pd.read_csv("merged_courses.tsv", delimiter="\t")

We can take a look at what it looks like:

In [3]:
dataset.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fee,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,True,\n\n3D visualisation and animation play a role...,September,\nFees\n\nPlease see the university website fo...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,True,\n\nBusinesses and governments rely on sound f...,September,"\nFees\n\nUK: £18,000 (Total) International: £...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,True,\n\nThe Analytical Toxicology MSc is a unique ...,See Course,\nFees\n\nPlease see the university website fo...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...
3,Entrepreneurship & Innovation - MSc,Trinity College Dublin,Trinity Business School,True,\n\nThis programme is designed to equip studen...,September,\nFees\n\nPlease see the university website fo...,MSc,1 year full-time,Dublin,Ireland,On Campus,https://www.findamasters.com//masters-degrees/...
4,Amputation and Prosthetic Rehabilitation MSc,University of Southampton,Faculty of Environmental and Life Sciences,True,\n\nEnhance your practice at a global top 100*...,September,\nFees\n\nPlease see the university website fo...,MSc,1 Year Full Time / 2-4 Years Part Time,Southampton,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...


#### Stemming with NLTK

We can use the NLTK library for stemming purposes:

In [4]:
from nltk.stem import *

Firstly, we remove all the rows that contain a NULL description:

In [5]:
dataset = dataset[dataset.description != '']

In [ ]:
dataset = dataset.dropna(subset=['description'])

As you may have noticed, the description cells include a lot of "\n" characters in them. So, in order to remove them, we use this, and take another look at the cleaner dataset:

In [6]:
# Replace '\n' with a space or empty string as you prefer
dataset['description'] = dataset['description'].str.replace('\n', ' ', regex=False)
dataset.description.head()

0      3D visualisation and animation play a role i...
1      Businesses and governments rely on sound fin...
2      The Analytical Toxicology MSc is a unique st...
3      This programme is designed to equip students...
4      Enhance your practice at a global top 100* u...
Name: description, dtype: object

##### Port Stemmer

One of the most common methods for stemming is Porter Stemmer:

In [8]:
porterstemmer = PorterStemmer()

We create a new column to house the stemmed version of the description column, called "descr_stem":

In [10]:
dataset['descr_stem'] = dataset.description.apply(lambda row: [porterstemmer.stem(word) for word in row.split(' ')])

We can see how the list created by splitting and stemming the description column of the 1st row looks like:

In [9]:
lst_porter = [porterstemmer.stem(word) for word in dataset.loc[0,'description'].split(' ')]
lst_porter

['',
 '',
 '3d',
 'visualis',
 'and',
 'anim',
 'play',
 'a',
 'role',
 'in',
 'mani',
 'areas,',
 'and',
 'the',
 'popular',
 'of',
 'these',
 'media',
 'just',
 'keep',
 'growing.',
 'digit',
 'anim',
 'provid',
 'the',
 'eye-catch',
 'special',
 'effect',
 'in',
 'the',
 '21st',
 "century'",
 'favourit',
 'film',
 'and',
 'televis',
 'shows;',
 '3d',
 'design',
 'is',
 'also',
 'essenti',
 'to',
 'everyday',
 'work',
 'in',
 'everyth',
 'from',
 'comput',
 'game',
 'development,',
 'onlin',
 'virtual',
 'world',
 'develop',
 'and',
 'industri',
 'design',
 'to',
 'marketing,',
 'product',
 'design',
 'and',
 "architecture.gcu'",
 'programm',
 'in',
 '3d',
 'design',
 'for',
 'virtual',
 'environ',
 'will',
 'help',
 'you',
 'develop',
 'the',
 'skill',
 'to',
 'thrive',
 'in',
 'a',
 'success',
 'career',
 'as',
 'a',
 'visual',
 'designer.',
 'the',
 'programm',
 'is',
 'practic',
 'and',
 'career-focused,',
 'orient',
 'toward',
 'current',
 'industri',
 'needs,',
 'technolog',
 '

##### SnowBall Stemmer

Another common method for stemming, is using the SnowBall stemmer:

In [11]:
snowstem = snowball.SnowballStemmer('english')

And then we can take a look at how different the results are between stemming with Porter Stemmer & SnowBall stemmer...
Apparantly, there's not much difference (at least in row #0)

In [12]:
lst_snow = [snowstem.stem(word) for word in dataset.loc[0,'description'].split(' ')]
lst_snow

['',
 '',
 '3d',
 'visualis',
 'and',
 'anim',
 'play',
 'a',
 'role',
 'in',
 'mani',
 'areas,',
 'and',
 'the',
 'popular',
 'of',
 'these',
 'media',
 'just',
 'keep',
 'growing.',
 'digit',
 'anim',
 'provid',
 'the',
 'eye-catch',
 'special',
 'effect',
 'in',
 'the',
 '21st',
 'centuri',
 'favourit',
 'film',
 'and',
 'televis',
 'shows;',
 '3d',
 'design',
 'is',
 'also',
 'essenti',
 'to',
 'everyday',
 'work',
 'in',
 'everyth',
 'from',
 'comput',
 'game',
 'development,',
 'onlin',
 'virtual',
 'world',
 'develop',
 'and',
 'industri',
 'design',
 'to',
 'marketing,',
 'product',
 'design',
 'and',
 'architecture.gcu',
 'programm',
 'in',
 '3d',
 'design',
 'for',
 'virtual',
 'environ',
 'will',
 'help',
 'you',
 'develop',
 'the',
 'skill',
 'to',
 'thrive',
 'in',
 'a',
 'success',
 'career',
 'as',
 'a',
 'visual',
 'designer.',
 'the',
 'programm',
 'is',
 'practic',
 'and',
 'career-focused,',
 'orient',
 'toward',
 'current',
 'industri',
 'needs,',
 'technolog',
 'an

#### Removing StopWords with NLTK

In order to create a search engine, we need to get rid of some words that don't give much informatrion and are quite redundant (stop words). 

In [13]:
import nltk
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amiralismac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
lst_stopwords = stopwords.words('english')

Here, we can take a look at all the stopwords included in the NLTK library:

In [16]:
lst_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

As you can see, words such as "I", "him", "wouldn't" and etc are not relevant, and are going to interfere with our search results.

Now, we can add another column to the dataset -- "descr_clean" -- that only contains the stemmed words that are NOT in the stopwords list:

In [17]:
dataset['descr_clean'] = dataset.description.apply(lambda row: [porterstemmer.stem(word) for word in row.split(' ') if not word in lst_stopwords])

Here's a quick look at the difference between the 2 columns we added at the end...

One -- "descr_stem" -- contains the stemmed version of all of the words.

But "descr_clean" has stopwords removed:

In [18]:
dataset.loc[0, ['descr_clean', 'descr_stem']]

descr_clean    [, , 3d, visualis, anim, play, role, mani, are...
descr_stem     [, , 3d, visualis, and, anim, play, a, role, i...
Name: 0, dtype: object

For example, the words "and" and "a" are removed.

#### Removing Punctuation with NLTK

Another step for preparing the description for the search engine, is to remove the punctuation.

In [19]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/amiralismac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
import string

As you can see, we don't need to include these charcters -- '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' -- in the searching process.

In [21]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

So, again we take a look at the description of the 1st row:

In [22]:
dataset.loc[0, 'description']

"  3D visualisation and animation play a role in many areas, and the popularity of these media just keeps growing. Digital animation provides the eye-catching special effects in the 21st century's favourite films and television shows; 3D design is also essential to everyday work in everything from computer games development, online virtual world development and industrial design to marketing, product design and architecture.GCU's programme in 3D Design for Virtual Environments will help you develop the skills to thrive in a successful career as a visual designer. The programme is practical and career-focused, oriented towards current industry needs, technology and practice. No prior knowledge of 3D design is required.  "

The NLTK's word_tokenizer is another powerful tool to split the words into a string from eachother and return the results as a list.

Instead of using the split() method and NLTK's punctuation package, we can use the word_tokenizer to split the words, and then remove the punctuation using the isalnum() method which checks whether a word contains AlphaNumeric characters or not...

Here's the result of using it on the 1st row's description column:

In [23]:
words = nltk.word_tokenize(dataset.loc[0, 'description'])
words

['3D',
 'visualisation',
 'and',
 'animation',
 'play',
 'a',
 'role',
 'in',
 'many',
 'areas',
 ',',
 'and',
 'the',
 'popularity',
 'of',
 'these',
 'media',
 'just',
 'keeps',
 'growing',
 '.',
 'Digital',
 'animation',
 'provides',
 'the',
 'eye-catching',
 'special',
 'effects',
 'in',
 'the',
 '21st',
 'century',
 "'s",
 'favourite',
 'films',
 'and',
 'television',
 'shows',
 ';',
 '3D',
 'design',
 'is',
 'also',
 'essential',
 'to',
 'everyday',
 'work',
 'in',
 'everything',
 'from',
 'computer',
 'games',
 'development',
 ',',
 'online',
 'virtual',
 'world',
 'development',
 'and',
 'industrial',
 'design',
 'to',
 'marketing',
 ',',
 'product',
 'design',
 'and',
 'architecture.GCU',
 "'s",
 'programme',
 'in',
 '3D',
 'Design',
 'for',
 'Virtual',
 'Environments',
 'will',
 'help',
 'you',
 'develop',
 'the',
 'skills',
 'to',
 'thrive',
 'in',
 'a',
 'successful',
 'career',
 'as',
 'a',
 'visual',
 'designer',
 '.',
 'The',
 'programme',
 'is',
 'practical',
 'and',
 '

We can check whether each word in the obtained list is or isn't an AlphaNumeric word...

In [24]:
for word_i in words:
    print(f"WORD: {word_i}\tisAlNum: {word_i.isalnum()}")


WORD: 3D	isAlNum: True
WORD: visualisation	isAlNum: True
WORD: and	isAlNum: True
WORD: animation	isAlNum: True
WORD: play	isAlNum: True
WORD: a	isAlNum: True
WORD: role	isAlNum: True
WORD: in	isAlNum: True
WORD: many	isAlNum: True
WORD: areas	isAlNum: True
WORD: ,	isAlNum: False
WORD: and	isAlNum: True
WORD: the	isAlNum: True
WORD: popularity	isAlNum: True
WORD: of	isAlNum: True
WORD: these	isAlNum: True
WORD: media	isAlNum: True
WORD: just	isAlNum: True
WORD: keeps	isAlNum: True
WORD: growing	isAlNum: True
WORD: .	isAlNum: False
WORD: Digital	isAlNum: True
WORD: animation	isAlNum: True
WORD: provides	isAlNum: True
WORD: the	isAlNum: True
WORD: eye-catching	isAlNum: False
WORD: special	isAlNum: True
WORD: effects	isAlNum: True
WORD: in	isAlNum: True
WORD: the	isAlNum: True
WORD: 21st	isAlNum: True
WORD: century	isAlNum: True
WORD: 's	isAlNum: False
WORD: favourite	isAlNum: True
WORD: films	isAlNum: True
WORD: and	isAlNum: True
WORD: television	isAlNum: True
WORD: shows	isAlNum: True
WO

Here's a list of all AlphaNumeric words in the above list. We can see that the punctuation has been removed:

In [25]:
[word_i for word_i in words if word_i.isalnum()]

['3D',
 'visualisation',
 'and',
 'animation',
 'play',
 'a',
 'role',
 'in',
 'many',
 'areas',
 'and',
 'the',
 'popularity',
 'of',
 'these',
 'media',
 'just',
 'keeps',
 'growing',
 'Digital',
 'animation',
 'provides',
 'the',
 'special',
 'effects',
 'in',
 'the',
 '21st',
 'century',
 'favourite',
 'films',
 'and',
 'television',
 'shows',
 '3D',
 'design',
 'is',
 'also',
 'essential',
 'to',
 'everyday',
 'work',
 'in',
 'everything',
 'from',
 'computer',
 'games',
 'development',
 'online',
 'virtual',
 'world',
 'development',
 'and',
 'industrial',
 'design',
 'to',
 'marketing',
 'product',
 'design',
 'and',
 'programme',
 'in',
 '3D',
 'Design',
 'for',
 'Virtual',
 'Environments',
 'will',
 'help',
 'you',
 'develop',
 'the',
 'skills',
 'to',
 'thrive',
 'in',
 'a',
 'successful',
 'career',
 'as',
 'a',
 'visual',
 'designer',
 'The',
 'programme',
 'is',
 'practical',
 'and',
 'oriented',
 'towards',
 'current',
 'industry',
 'needs',
 'technology',
 'and',
 'pract

So, this time we revise the "descr_clean" column: We only want the words that have been stemmed and are not stopwords.

In [26]:
dataset['descr_clean'] = dataset.description.apply(lambda row: [porterstemmer.stem(word) for word in nltk.word_tokenize(row) if not word in lst_stopwords and word.isalnum()])
dataset['descr_clean'].head

<bound method NDFrame.head of 0       [3d, visualis, anim, play, role, mani, area, p...
1       [busi, govern, reli, sound, financi, knowledg,...
2       [the, analyt, toxicolog, msc, uniqu, studi, co...
3       [thi, programm, design, equip, student, specia...
4       [enhanc, practic, global, top, 100, understand...
                              ...                        
5995    [allergi, increas, global, health, problem, th...
5996    [thi, new, postgradu, certif, allergi, clinic,...
5997    [we, world, allergi, organis, centr, professio...
5998    [a, research, degre, open, door, work, researc...
5999    [programm, descriptionthi, programm, allow, ex...
Name: descr_clean, Length: 6000, dtype: object>

In [27]:
dataset.loc[0, 'descr_clean']

['3d',
 'visualis',
 'anim',
 'play',
 'role',
 'mani',
 'area',
 'popular',
 'media',
 'keep',
 'grow',
 'digit',
 'anim',
 'provid',
 'special',
 'effect',
 '21st',
 'centuri',
 'favourit',
 'film',
 'televis',
 'show',
 '3d',
 'design',
 'also',
 'essenti',
 'everyday',
 'work',
 'everyth',
 'comput',
 'game',
 'develop',
 'onlin',
 'virtual',
 'world',
 'develop',
 'industri',
 'design',
 'market',
 'product',
 'design',
 'programm',
 '3d',
 'design',
 'virtual',
 'environ',
 'help',
 'develop',
 'skill',
 'thrive',
 'success',
 'career',
 'visual',
 'design',
 'the',
 'programm',
 'practic',
 'orient',
 'toward',
 'current',
 'industri',
 'need',
 'technolog',
 'practic',
 'no',
 'prior',
 'knowledg',
 '3d',
 'design',
 'requir']

#### SpaCy

One powerful and more modern package that is commonly used is SpaCy. We won't use SpaCy a lot, but we only want to showcase its abilities.

In [28]:
import spacy

In [29]:
nlp = spacy.load("en_core_web_sm")

In [30]:
doc = nlp(dataset.loc[0, 'description'])

In [31]:
doc

  3D visualisation and animation play a role in many areas, and the popularity of these media just keeps growing. Digital animation provides the eye-catching special effects in the 21st century's favourite films and television shows; 3D design is also essential to everyday work in everything from computer games development, online virtual world development and industrial design to marketing, product design and architecture.GCU's programme in 3D Design for Virtual Environments will help you develop the skills to thrive in a successful career as a visual designer. The programme is practical and career-focused, oriented towards current industry needs, technology and practice. No prior knowledge of 3D design is required.  

We can iterate through each and every entity of our "doc" variable. And we can see each of their characteristics, such as:

.text: The word itself.

.start_char: How many characters before the word begins.

.end_char: How many characters before the word ends.

.label_: A categorized labeling specific to SpaCy itself.

In [32]:
for ent_i in doc.ents:
    print(f"TEXT: {ent_i.text}, START_CHAR: {ent_i.start_char}, END_CHAR: {ent_i.end_char}, LABEL_: {ent_i.label_}")

TEXT: Digital, START_CHAR: 114, END_CHAR: 121, LABEL_: ORG
TEXT: the 21st century's, START_CHAR: 177, END_CHAR: 195, LABEL_: DATE
TEXT: GCU, START_CHAR: 427, END_CHAR: 430, LABEL_: ORG
TEXT: 3D Design for Virtual Environments, START_CHAR: 446, END_CHAR: 480, LABEL_: ORG


And we can also look at some other methods of each token:

In [33]:
for token in doc:
    print(token, token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

         SPACE _SP dep    False False
3D 3D 3d ADJ JJ amod dX False False
visualisation visualisation visualisation NOUN NN nsubj xxxx True False
and and and CCONJ CC cc xxx True True
animation animation animation NOUN NN conj xxxx True False
play play play VERB VBP ROOT xxxx True False
a a a DET DT det x True True
role role role NOUN NN dobj xxxx True False
in in in ADP IN prep xx True True
many many many ADJ JJ amod xxxx True True
areas areas area NOUN NNS pobj xxxx True False
, , , PUNCT , punct , False False
and and and CCONJ CC cc xxx True True
the the the DET DT det xxx True True
popularity popularity popularity NOUN NN nsubj xxxx True False
of of of ADP IN prep xx True True
these these these DET DT det xxxx True True
media media medium NOUN NNS pobj xxxx True False
just just just ADV RB advmod xxxx True True
keeps keeps keep VERB VBZ conj xxxx True False
growing growing grow VERB VBG xcomp xxxx True False
. . . PUNCT . punct . False False
Digital Digital digital ADJ JJ amod Xxxx

As mentioned earlier, we will not use SpaCy for our search engine.

### 2.0.1 Preprocessing the fees column

Moreover, we want the field fees to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a float column renamed fees (CHOSEN COMMON CURRENCY).

Now, let's take a look at the first ten rows of the fees column:

In [35]:
for fee_i in (dataset.fee[:10]):
    print(fee_i)


Fees

Please see the university website for further information on fees for this course.



Fees

UK: £18,000 (Total) International: £34,750 (Total)



Fees

Please see the university website for further information on fees for this course.



Fees

Please see the university website for further information on fees for this course.



Fees

Please see the university website for further information on fees for this course.



Fees

Part time - £7,050



Fees

Please see the university website for further information on fees for this course.



Fees

Please see the university website for further information on fees for this course.



Fees

Please see the university website for further information on fees for this course.



Fees

Our tuition fee for UK students starting full-time study in 2024/25 is £10,310 for the course.  Please see the university website for further information on fees for this course.




And this is just the fees column of the 1st row:

In [37]:
dataset.fee.iloc[0]

'\nFees\n\nPlease see the university website for further information on fees for this course.\n\n'

We can see that sometimes no figure has been mentioned. And at other times, several figures may have been mentioned. So, we should take care of that.

##### Making API requests to exchangeratesapi.io for Conversion Rates

Now, we want to use freecurrencyapi API to gather data about the latest exchange rates:

In [38]:
pip install git+https://github.com/everapihq/freecurrencyapi-python.git

  Cloning https://github.com/everapihq/freecurrencyapi-python.git to /private/var/folders/zk/kfcj5gj53s9dcj21xxz5w1q00000gn/T/pip-req-build-y0rx0lu7
  Running command git clone --filter=blob:none --quiet https://github.com/everapihq/freecurrencyapi-python.git /private/var/folders/zk/kfcj5gj53s9dcj21xxz5w1q00000gn/T/pip-req-build-y0rx0lu7
  Resolved https://github.com/everapihq/freecurrencyapi-python.git to commit c32c4dc9df5de4cf0d4940c0c27efc5fc8473bdf
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


We use our own personal API code to fetch data.

In [39]:
import freecurrencyapi
client = freecurrencyapi.Client('fca_live_gN70PpWZqBXZc2mLgxw9mcWxOAnrO1el551fuHET')

We can also check the status of our connection and account info:

In [40]:
print(client.status())

{'account_id': 247677987040923648, 'quotas': {'month': {'total': 5000, 'used': 11, 'remaining': 4989}, 'grace': {'total': 0, 'used': 0, 'remaining': 0}}}


The line below gives us a dictionary of the latest exchange rates:

In [41]:
rates = client.latest()
print(rates)

{'data': {'AUD': 1.524270282, 'BGN': 1.7824602166, 'BRL': 4.8530606143, 'CAD': 1.3724201969, 'CHF': 0.8845901093, 'CNY': 7.1691813138, 'CZK': 22.3758329505, 'DKK': 6.8112410044, 'EUR': 0.9136300929, 'GBP': 0.7995401475, 'HKD': 7.7921410933, 'HRK': 7.0430208018, 'HUF': 346.6651834333, 'IDR': 15428.155505614, 'ILS': 3.7082405579, 'INR': 83.3152225189, 'ISK': 139.2661947373, 'JPY': 148.364957359, 'KRW': 1286.059432812, 'MXN': 17.1055118772, 'MYR': 4.6669809279, 'NOK': 10.692161539, 'NZD': 1.655790238, 'PHP': 55.3833879628, 'PLN': 3.9712405892, 'RON': 4.5418206452, 'RUB': 88.2323940745, 'SEK': 10.4392714591, 'SGD': 1.3375901567, 'THB': 35.1690249021, 'TRY': 28.7827055726, 'USD': 1, 'ZAR': 18.3249629199}}


##### Extracting & Converting Fees

REGEX is a powerful tool we can use to identify where and how some characters have been used in a string. This helps us identify the fee figures.

So we need to import it at first:

In [44]:
import re

This is a function defined to extract & convert the fees.

In [42]:
def extract_and_convert_fees(row, rates):
    max_fee_usd = 0  # Initialize the maximum fee variable
    # Define the currency symbols which will be used to identify the currency in the regex pattern.
    currency_symbols = {
        'USD': r'\$',
        'EUR': r'€',
        'JPY': r'¥',
        'GBP': r'£',
        'AUD': r'A\$',
        'CAD': r'C\$',
        'CHF': r'Fr',
        'CNY': r'¥',  # The same symbol as JPY, might need additional context to differentiate
        'HKD': r'HK\$',
        'NZD': r'NZ\$'
    }

    # Regex to find numbers - this is a simplistic pattern
    for currency, symbol in currency_symbols.items():
        pattern = rf'{symbol}(\d+(?:,\d{{3}})*(\.\d+)?)'
        matches = re.findall(pattern, row)
        for match in matches:
            # Remove commas and convert to float
            num = float(match[0].replace(',', ''))
            # Multiply by the conversion rate to get the amount in USD
            if currency in rates['data']:
                num *= rates['data'][currency]
                max_fee_usd = max(max_fee_usd, num)

    return max_fee_usd

We can see an example usage, where the function correctly identifies the two amounts (15K€ & 13K£). Converts them both to the USD$ and finds the max amount and returns it:

In [43]:
# Example usage:
row = "The tuition fee is €15,000 or £13,000 based on the current exchange rates."

# Calculate the fee in USD
fee_in_usd = extract_and_convert_fees(row, rates)
print(f"The fee in USD is: {fee_in_usd}")

The fee in USD is: 13704.4513935


Or, as another example, we can look at fee figure converted to USD$ at row 5:

In [46]:
# This will apply the function to the first row's 'fee' value only
fee_in_usd = extract_and_convert_fees(dataset['fee'].iloc[5], rates)
print(f"The fee in USD is: {fee_in_usd}")


The fee in USD is: 5636.758039875001


In [48]:
import numpy as np

We should pay attention to the fact that some rows will return NaN values. So, when creating the new "fees_in_usd" coumn, we need to take care of them as well:

In [49]:

# Apply the function to every row, handling floats by converting them to strings
dataset['fees_in_usd'] = dataset['fee'].apply(lambda row: extract_and_convert_fees(str(row), rates) if not pd.isnull(row) else np.nan)


Now, we can see that in places where no fee figure has been mentioned, the amount represented in our new last column is 0.0

In [50]:
dataset.head(10)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fee,modality,duration,city,country,administration,url,descr_stem,descr_clean,fees_in_usd
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,True,3D visualisation and animation play a role i...,September,\nFees\n\nPlease see the university website fo...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , 3d, visualis, and, anim, play, a, role, i...","[3d, visualis, anim, play, role, mani, area, p...",0.000000
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,True,Businesses and governments rely on sound fin...,September,"\nFees\n\nUK: £18,000 (Total) International: £...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , busi, and, govern, reli, on, sound, finan...","[busi, govern, reli, sound, financi, knowledg,...",27784.020126
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,True,The Analytical Toxicology MSc is a unique st...,See Course,\nFees\n\nPlease see the university website fo...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , the, analyt, toxicolog, msc, is, a, uniqu...","[the, analyt, toxicolog, msc, uniqu, studi, co...",0.000000
3,Entrepreneurship & Innovation - MSc,Trinity College Dublin,Trinity Business School,True,This programme is designed to equip students...,September,\nFees\n\nPlease see the university website fo...,MSc,1 year full-time,Dublin,Ireland,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , thi, programm, is, design, to, equip, stu...","[thi, programm, design, equip, student, specia...",0.000000
4,Amputation and Prosthetic Rehabilitation MSc,University of Southampton,Faculty of Environmental and Life Sciences,True,Enhance your practice at a global top 100* u...,September,\nFees\n\nPlease see the university website fo...,MSc,1 Year Full Time / 2-4 Years Part Time,Southampton,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , enhanc, your, practic, at, a, global, top...","[enhanc, practic, global, top, 100, understand...",0.000000
5,Anaesthesia and Perioperative Science MSc,University College London,Division of Surgery and Interventional Science,False,Register your interest in graduate study at ...,"March, September","\nFees\n\nPart time - £7,050\n\n",MSc,2 years part time,London,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , regist, your, interest, in, graduat, stud...","[regist, interest, graduat, studi, uclth, msc,...",5636.758040
6,Analogue and Digital Integrated Circuit Desi...,Imperial College London,Electrical and Electronic Engineering,True,Our MSc in Analogue and Digital Integrated C...,October,\nFees\n\nPlease see the university website fo...,MSc,1 year full-time,London,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , our, msc, in, analogu, and, digit, integr...","[our, msc, analogu, digit, integr, circuit, ai...",0.000000
7,Analytical Bioscience (MSc/PGDip),"Birkbeck, University of London",School of Natural Sciences,True,Our analytical bioscience postgraduate cours...,"October, January",\nFees\n\nPlease see the university website fo...,"MSc, PGDip",1 year full-time or 2 years part-time,London,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , our, analyt, bioscienc, postgradu, cours,...","[our, analyt, bioscienc, postgradu, cours, pro...",0.000000
8,Analytical Chemistry (MSc),Kingston University,"Faculty of Health, Science, Social Care and Ed...",True,You will study good measurement and scientif...,"September, January",\nFees\n\nPlease see the university website fo...,MSc,"1 year full time, 2 years full time including ...",London,United Kingdom,On Campus,https://www.f

### 2.1. Conjunctive query

For the first version of the search engine, we narrowed our interest to the description of each course. It means that you will evaluate queries only concerning the course's description.

#### 2.1.1 Create your index!

Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
where document_i is the id of a document that contains that specific word.

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth thinking about storing it in a separate file and loading it in memory when needed.

Let's take a look again at the the "descr_clean" column:

In [51]:
dataset.descr_clean

0       [3d, visualis, anim, play, role, mani, area, p...
1       [busi, govern, reli, sound, financi, knowledg,...
2       [the, analyt, toxicolog, msc, uniqu, studi, co...
3       [thi, programm, design, equip, student, specia...
4       [enhanc, practic, global, top, 100, understand...
                              ...                        
5995    [allergi, increas, global, health, problem, th...
5996    [thi, new, postgradu, certif, allergi, clinic,...
5997    [we, world, allergi, organis, centr, professio...
5998    [a, research, degre, open, door, work, researc...
5999    [programm, descriptionthi, programm, allow, ex...
Name: descr_clean, Length: 6000, dtype: object

In order to create the vocabulary, we can use either of these 2 approaches:

##### Approach 1: Using Python Sets

We can define an empty set at first:

In [52]:
vocabulary = set()

And for each word in each row, we can consider adding them to the set:

In [56]:
dataset.descr_clean.apply(lambda row: [vocabulary.add(word) for word in row])

0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5995    [None, None, None, None, None, None, None, Non...
5996    [None, None, None, None, None, None, None, Non...
5997    [None, None, None, None, None, None, None, Non...
5998    [None, None, None, None, None, None, None, Non...
5999    [None, None, None, None, None, None, None, Non...
Name: descr_clean, Length: 6000, dtype: object

Obviously, we'll see the words that have been used repeatedly only once in the set:

In [57]:
vocabulary

{'addit',
 'not',
 'heatwav',
 'manag',
 '1998',
 'cp2',
 'middlesex',
 'constrain',
 'managementeduc',
 'speaker',
 'nurs',
 'both',
 'prematur',
 'sussex',
 'investor',
 'pluralist',
 'color',
 'makeracadem',
 'import',
 'tecton',
 'began',
 'macrophag',
 'imm',
 'seventh',
 'democratis',
 'undergon',
 'celci',
 'skills28',
 'adjac',
 'fix',
 'economicsstud',
 'designour',
 'standout',
 'marxist',
 'command',
 'gcp',
 'speakersmasterclassesresearch',
 'instil',
 'wellcom',
 'audiologist',
 'andsolv',
 'nsirc',
 'nanoengin',
 'einer',
 'tibco',
 'justiceand',
 'julien',
 'dwarf',
 'intelligenceth',
 'structur',
 'timeif',
 '21st',
 'style',
 'apm',
 'oversight',
 'managementar',
 'esct',
 'datamin',
 'nich',
 'geotechn',
 'subsequ',
 'wight',
 'kann',
 'hla',
 'employersal',
 'good',
 'ca',
 'presentationth',
 'imagingmed',
 'protein',
 'unemploy',
 'prehospit',
 'particl',
 '800',
 'doel',
 'implantolog',
 'socioeconom',
 '35',
 'morew',
 'grappl',
 'degreether',
 'resolut',
 'introd

##### Approach 2: Using Python Collections

Another approach is by using the counter and reduce packages:

In [58]:
from collections import Counter
from functools import reduce

In [59]:
vocabulary_alt = Counter(reduce(lambda x, y: x + y, dataset.descr_clean.values)).keys()

In [60]:
vocabulary_alt

dict_keys(['3d', 'visualis', 'anim', 'play', 'role', 'mani', 'area', 'popular', 'media', 'keep', 'grow', 'digit', 'provid', 'special', 'effect', '21st', 'centuri', 'favourit', 'film', 'televis', 'show', 'design', 'also', 'essenti', 'everyday', 'work', 'everyth', 'comput', 'game', 'develop', 'onlin', 'virtual', 'world', 'industri', 'market', 'product', 'programm', 'environ', 'help', 'skill', 'thrive', 'success', 'career', 'visual', 'the', 'practic', 'orient', 'toward', 'current', 'need', 'technolog', 'no', 'prior', 'knowledg', 'requir', 'busi', 'govern', 'reli', 'sound', 'financi', 'underpin', 'strategi', 'cours', 'profession', 'advanc', 'modern', 'theori', 'account', 'control', 'well', 'understand', 'organis', 'cover', 'fundament', 'topic', 'corpor', 'financ', 'report', 'option', 'modul', 'allow', 'specialis', 'rang', 'intern', 'forens', 'you', 'abl', 'appli', 'learn', 'case', 'studi', 'simul', 'test', 'genuin', 'scenario', 'analyt', 'toxicolog', 'msc', 'uniqu', 'integr', 'theoret', 'a

##### Assign Unique ID to each item in the Vocab

In order to make the inverted matrix, we need to assign INT values to each term in the Vocab...

This can be done in 2 ways:

###### Approach 1: Using For-Loop

In [61]:
index = {}
unique_id = 1
for word in list(vocabulary_alt):
  index[unique_id] = word
  unique_id+=1

In [62]:
index

{1: '3d',
 2: 'visualis',
 3: 'anim',
 4: 'play',
 5: 'role',
 6: 'mani',
 7: 'area',
 8: 'popular',
 9: 'media',
 10: 'keep',
 11: 'grow',
 12: 'digit',
 13: 'provid',
 14: 'special',
 15: 'effect',
 16: '21st',
 17: 'centuri',
 18: 'favourit',
 19: 'film',
 20: 'televis',
 21: 'show',
 22: 'design',
 23: 'also',
 24: 'essenti',
 25: 'everyday',
 26: 'work',
 27: 'everyth',
 28: 'comput',
 29: 'game',
 30: 'develop',
 31: 'onlin',
 32: 'virtual',
 33: 'world',
 34: 'industri',
 35: 'market',
 36: 'product',
 37: 'programm',
 38: 'environ',
 39: 'help',
 40: 'skill',
 41: 'thrive',
 42: 'success',
 43: 'career',
 44: 'visual',
 45: 'the',
 46: 'practic',
 47: 'orient',
 48: 'toward',
 49: 'current',
 50: 'need',
 51: 'technolog',
 52: 'no',
 53: 'prior',
 54: 'knowledg',
 55: 'requir',
 56: 'busi',
 57: 'govern',
 58: 'reli',
 59: 'sound',
 60: 'financi',
 61: 'underpin',
 62: 'strategi',
 63: 'cours',
 64: 'profession',
 65: 'advanc',
 66: 'modern',
 67: 'theori',
 68: 'account',
 69:

###### Approach 2: Using Pandas DataFrames

We create a Pandas DataFrame called "terms". Its first column will be named "term" and it'll simply house the terms of our vocab.

In [63]:
terms = pd.DataFrame(data=list(vocabulary_alt), columns=['term'])

In [64]:
terms

,term
0,3d
1,visualis
2,anim
3,play
4,role
...,...
8771,recherch
8772,mathématiqu
8773,allerg
8774,slaveri


##### Creating the Inverted Index 

Question: Which documents (from 0 to 5999) contain the word "3D" in them?
PS: terms.loc[0, "term"] is "3D"

In [66]:
dataset.descr_clean.apply(lambda row: terms.loc[0, "term"] in row)

0        True
1       False
2       False
3       False
4       False
        ...  
5995    False
5996    False
5997    False
5998    False
5999    False
Name: descr_clean, Length: 6000, dtype: bool

Using a mask to see exactly which documents contain the word "3D"

In [67]:
dataset.index[dataset.descr_clean.apply(lambda row: terms.loc[0, "term"] in row)]

Index([   0,  923, 1592, 2033, 2036, 2037, 3239, 3339, 3691, 4401, 4467, 5309,
       5346, 5357, 5868],
      dtype='int64')

Again, we use the same mask, but this time, in order to see courseName and description

In [68]:
dataset[["courseName", "description"]][dataset.descr_clean.apply(lambda row: terms.loc[0, "term"] in row)]

,courseName,description
0,3D Design for Virtual Environments - MSc,3D visualisation and animation play a role i...
923,Chemistry - MPhil/MSc (Research),Our research groups are involved in projects...
1592,Creative Technologies (MSc),Combine your passion for creative art and de...
2033,Digital Anthropology MSc,Register your interest in graduate study at ...
2036,Digital Audio Engineering - MSc,"On our MSc in Digital Audio Engineering, you..."
2037,Digital Audio Engineering - MSc,"On our MSc in Digital Audio Engineering, you..."
3239,Game Engineering - MSc,UCA's new MSc Game Engineering course aims t...
3339,Geospatial Sciences (Building Information Mo...,Register your interest in graduate study at ...
3691,Physics - MSc,This course is for you if you’re interested ...
4401,3D Computer Games Design - MSc,Get a flying start in the games industry. If...


The only reason that we use the "tqdm.notebook" is to use the "progress_apply" method later:

In [69]:
from tqdm.notebook import tqdm

tqdm.pandas()

So now, we want to add another column to the "terms" dataframe -- called "reverse". It contains a list of all the rows that include that specific term/word.

In [70]:

terms['reverse'] = terms.term.progress_apply(lambda item: list(dataset.loc[dataset.descr_clean.apply(lambda row: item in row)].index))

  0%|          | 0/8776 [00:00<?, ?it/s]

Let's take a look at the top 10:

In [72]:
terms.loc[0:9]

,term,reverse
0,3d,"[0, 923, 1592, 2033, 2036, 2037, 3239, 3339, 3..."
1,visualis,"[0, 78, 306, 323, 470, 471, 474, 738, 754, 796..."
2,anim,"[0, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34..."
3,play,"[0, 34, 42, 43, 66, 103, 249, 250, 254, 357, 3..."
4,role,"[0, 5, 31, 33, 34, 43, 46, 58, 103, 107, 119, ..."
5,mani,"[0, 33, 36, 49, 79, 94, 95, 99, 102, 109, 111,..."
6,area,"[0, 4, 11, 13, 18, 36, 47, 51, 53, 71, 75, 84,..."
7,popular,"[0, 322, 532, 1047, 1164, 1165, 1166, 1519, 15..."
8,media,"[0, 27, 54, 196, 284, 363, 364, 447, 476, 546,..."
9,keep,"[0, 119, 396, 462, 530, 619, 686, 687, 1359, 1..."


For easier use, let's save our inverted index into a csv file:

In [73]:
terms.to_csv('inverted_index.csv', index=False)

#### Executing the Query

Finally, we can define a very simple search engine function -- "execute_query":

In [80]:
def execute_query(query, inverted_index, dataset):
    # Split the query into terms and stem each term
    query_terms = [porterstemmer.stem(word) for word in query.lower().split()]
    
    # Retrieve the list of document indices for each query term
    list_of_documents = [set(inverted_index[inverted_index['term'] == term_i]['reverse'].values[0])
                         for term_i in query_terms if term_i in inverted_index['term'].values]
    
    # Intersect the document lists to find documents that contain all query terms
    if list_of_documents:
        docs_containing_all_terms = set.intersection(*list_of_documents)
        # Convert set to list before using it as an indexer
        docs_containing_all_terms = list(docs_containing_all_terms)
    else:
        docs_containing_all_terms = []

    results = dataset.loc[docs_containing_all_terms, ['courseName', 'universityName', 'description', 'url']]
    
    return results.reset_index(drop=True)


Example usage of the search engine:

In [81]:
# Example usage:
sample_query = "advanced knowledge"
query_results = execute_query(sample_query, terms, dataset)
(query_results)

,courseName,universityName,description,url
0,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fin...,https://www.findamasters.com//masters-degrees/...
1,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique st...,https://www.findamasters.com//masters-degrees/...
2,Digital Design and Branding MSc,Brunel University London,Ask BrunelOur Digital Design and Branding MS...,https://www.findamasters.com//masters-degrees/...
3,Digital Design and Manufacture MSc,University of Edinburgh,Programme descriptionWith the transition to ...,https://www.findamasters.com//masters-degrees/...
4,Stroke Medicine MSc,Learna | Diploma MSc,Postgraduate training in stroke care is limi...,https://www.findamasters.com//masters-degrees/...
...,...,...,...,...
466,International Business - MSc,University of Glasgow,International Business will provide you with...,https://www.findamasters.com//masters-degrees/...
467,Diabetes Practice - MSc,Cardiff University,Why study this courseThe MSc Diabetes Practi...,https://www.findamasters.com//masters-degrees/...
468,Diabetes Practice MSc/PGDip/PGCert,Swansea University,Diabetes Practice Course OverviewIf you are ...,https://www.findamasters.com//masters-degrees/...
469,Diagnostic Imaging - MSc,Glasgow Caledonian University,Designed to train a new generation of leader...,https://www.findamasters.com//masters-degrees/...


Another example usage (more specific):

In [82]:
# Example usage:
sample_query = "financial management"
query_results = execute_query(sample_query, terms, dataset)
(query_results)

,courseName,universityName,description,url
0,Master in Global Management at ESMT Berlin,ESMT Berlin,#1 in Germany and #10 GloballyThe ESMT Berli...,https://www.findamasters.com//masters-degrees/...
1,Accounting and Finance - MSc,Bangor University,This degree programme provides the opportuni...,https://www.findamasters.com//masters-degrees/...
2,MSc Finance and Investment Management,University of Liverpool,A career in finance and investment managemen...,https://www.findamasters.com//masters-degrees/...
3,Master in Health Care Management,Harvard University,"Today, leading a health care organization ta...",https://www.findamasters.com//masters-degrees/...
4,FinTech Management - MSc,Ulster University,Gain a solid grounding in key aspects of fin...,https://www.findamasters.com//masters-degrees/...
...,...,...,...,...
238,Master in Global Logistics and Supply Chain ...,Kuehne Logistics University - KLU,Our MSc in Global Logistics and Supply Chain...,https://www.findamasters.com//masters-degrees/...
239,Master of Finance,York University (Canada),The “must-have” degree for specialized finan...,https://www.findamasters.com//masters-degrees/...
240,Financial Technology (Fintech) MSc,University of Exeter,OverviewGain an education in finance and eco...,https://www.findamasters.com//masters-degrees/...
241,Construction Project Management MSc,University of Portsmouth,OverviewEstablish yourself in the global con...,https://www.findamasters.com//masters-degrees/...


We can see it works pretty well.

### 2.2 Conjunctive query & Ranking score

#### TF-IDF

Now, another way to define our search engine is to use the TF-IDF score:

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [84]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1

In [85]:
results = tfidf.fit_transform(dataset.descr_clean)

/opt/homebrew/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [86]:
results

<6000x8776 sparse matrix of type '<class 'numpy.float64'>'
	with 240878 stored elements in Compressed Sparse Row format>

In [87]:
result_dense = results.todense()

In [88]:
result_dense

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Finally, we can make a Pandas DataFrame using the score in the "descr_clean" column:

In [89]:
tfidf_data = pd.DataFrame(result_dense.tolist(), index=dataset.index, columns=tfidf.get_feature_names_out())

In [90]:
len(tfidf_data)

6000

In [91]:
tfidf_data

,1,10,100,1000,104k,11,11th,12,120,125k,...,zoonosi,zoonot,zu,zudem,zum,zur,zurich,zwingen,école,ísafjörður
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.152693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5996,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5997,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5998,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Just to get a sense of what the dataframe looks like, we can take a look at some of its middle columns

In [92]:
tfidf_data.iloc[:, 1150:1165]

,bremerhaven,brew,breweri,brexit,bric,bridg,brief,briefconceptu,briefth,bright,brightest,brighton,brilliant,brim,bring
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
tfidf_data.loc[terms.term == "3d"]["3d"]

0    0.569285
Name: 3d, dtype: float64

We know that in index=0, the word "3d" has been used a lot. So, let's take a look at how many times it has been used:

In [94]:
Counter(dataset.loc[0, 'descr_clean'])['3d']

4

We know that in index=1, the word "govern" has been used once. So, let's take a look at that:

In [99]:
Counter(dataset.loc[1, 'descr_clean'])['govern']

1

If we look for the word "finance" in tfidf_data, we can see that it has a 0.0 TFIDF score in most of the documents:

In [100]:
# Obviously we cannot mention "finance" exactly as is, but we have to stemm it first into "financ"
tfidf_data.financ

0       0.000000
1       0.218261
2       0.000000
3       0.146376
4       0.000000
          ...   
5995    0.000000
5996    0.000000
5997    0.000000
5998    0.000000
5999    0.000000
Name: financ, Length: 6000, dtype: float64

So, let's take a look at all the documents in which the word "finance" has been used:

In [101]:
tfidf_data.loc[tfidf_data.financ > 0, ['financ']]

,financ
1,0.218261
3,0.146376
91,0.080630
93,0.105536
100,0.113364
...,...
5845,0.233945
5894,0.115720
5932,0.115054
5960,0.080887


In [102]:
dataset[tfidf_data.financ > 0][["courseName", "description"]]

,courseName,description
1,Accounting and Finance - MSc,Businesses and governments rely on sound fin...
3,Entrepreneurship & Innovation - MSc,This programme is designed to equip students...
91,"Environmental, Social, Governance (ESG) Mana...",More than ever ESG is essential for business...
93,Applied Data Science - MSc,Our Applied Data Science Masters programme i...
100,Applied data science for banking and finance...,The Master’s degree of Applied Data Science ...
...,...,...
5845,Economics and Finance - MSc,This course provides an opportunity to combi...
5894,Advanced Technology for Financial Computing ...,Programme descriptionThis programme will pro...
5932,Aerospace Engineering (MSc),This course provides a strategic and practic...
5960,Agricultural Economics,"Food crises, commodity price booms and busts..."


We can see that the word "you" has been used in a lot of documents, but with a relativley low TFIDF score:

In [103]:
tfidf_data.loc[tfidf_data["you"] > 0]["you"]

1       0.081938
8       0.122906
22      0.077745
31      0.180371
36      0.069929
          ...   
5965    0.160136
5973    0.051741
5984    0.075699
5997    0.063938
5999    0.045656
Name: you, Length: 1160, dtype: float64

We can also create a "descr_len" column and add it at the end of our dataset. It shows the number of terms/words in each decription cell.

In [107]:
dataset['descr_len'] = dataset.descr_clean.apply(lambda row: len(row))

In [108]:
dataset.loc[[1, 2, 9, 12, 14], 'descr_len']

1     55
2     27
9     14
12     6
14    63
Name: descr_len, dtype: int64

### Cosine Similarity

A powerful measure to use in order to create our second search engine is Cosine Similarity. 

In [109]:
from sklearn.metrics.pairwise import cosine_similarity

We create a DataFrame that shows the Cosine Similarity between each and every of our 6000 documents.

In [110]:
cossim_data = pd.DataFrame(cosine_similarity(tfidf_data), index=dataset.index, columns=dataset.index)

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [111]:
cossim_data

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,1.000000,0.038019,0.018212,0.049540,0.027415,0.032051,0.072937,0.027037,0.025036,0.033349,...,0.070460,0.052177,0.060756,0.016675,0.000000,0.032074,0.012516,0.030333,0.030527,0.007140
1,0.038019,1.000000,0.094259,0.096407,0.019319,0.032667,0.021137,0.042857,0.038957,0.102501,...,0.085565,0.028854,0.023642,0.024240,0.007203,0.043341,0.015176,0.041172,0.060891,0.048022
2,0.018212,0.094259,1.000000,0.021936,0.026290,0.042686,0.036890,0.118790,0.008484,0.127204,...,0.051069,0.034747,0.029922,0.028111,0.019928,0.036409,0.039327,0.023576,0.048142,0.010864
3,0.049540,0.096407,0.021936,1.000000,0.011052,0.016113,0.018923,0.013882,0.018241,0.012421,...,0.077063,0.028621,0.037369,0.020286,0.004974,0.036506,0.043815,0.019078,0.055987,0.014047
4,0.027415,0.019319,0.026290,0.011052,1.000000,0.058880,0.009719,0.007095,0.025373,0.025290,...,0.005714,0.022339,0.017862,0.005316,0.002312,0.078720,0.057107,0.103843,0.112800,0.011415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.032074,0.043341,0.036409,0.036506,0.078720,0.034173,0.013846,0.029106,0.020673,0.038491,...,0.017626,0.034567,0.033332,0.005534,0.012587,1.000000,0.297359,0.447047,0.060577,0.007407
5996,0.012516,0.015176,0.039327,0.043815,0.057107,0.066077,0.000000,0.042990,0.006433,0.013125,...,0.015428,0.051301,0.023282,0.005872,0.006416,0.297359,1.000000,0.301451,0.082535,0.006996
5997,0.030333,0.041172,0.023576,0.019078,0.103843,0.052580,0.014557,0.015810,0.034284,0.000000,...,0.021736,0.009945,0.013663,0.012939,0.014338,0.447047,0.301451,1.000000,0.127476,0.002919
5998,0.030527,0.060891,0.048142,0.055987,0.112800,0.121768,0.029364,0.056388,0.042876,0.010908,...,0.032740,0.023966,0.004690,0.021776,0.038178,0.060577,0.082535,0.127476,1.000000,0.056567


For example we can look at all the instances (rows) in which there's a non-zero TF-IDF score for the word "design":

In [112]:
tfidf_data.loc[tfidf_data.design > 0].index

Index([   0,    3,    6,    7,   15,   16,   17,   19,   24,   25,
       ...
       5982, 5984, 5985, 5986, 5987, 5989, 5990, 5991, 5992, 5995],
      dtype='int64', length=1592)

And then we can look at the cosine similarity btwn those documents/rows and every other document:

In [113]:
cossim_data[tfidf_data.loc[tfidf_data.design > 0.1].index]

,0,6,61,76,77,85,103,135,147,161,...,5868,5884,5892,5905,5921,5944,5978,5984,5986,5992
0,1.000000,0.072937,0.074057,0.095088,0.086102,0.116848,0.094839,0.049948,0.090475,0.045330,...,0.339156,0.082803,0.108420,0.070308,0.065659,0.086240,0.088337,0.112924,0.089741,0.060756
1,0.038019,0.021137,0.100035,0.053396,0.059820,0.074271,0.043321,0.052476,0.068788,0.025999,...,0.087619,0.064105,0.083016,0.038224,0.103496,0.080534,0.116896,0.078784,0.052316,0.023642
2,0.018212,0.036890,0.131352,0.048856,0.044283,0.028325,0.057176,0.017569,0.024269,0.035880,...,0.052236,0.082656,0.064434,0.104520,0.014280,0.053015,0.044795,0.048245,0.037336,0.029922
3,0.049540,0.018923,0.092642,0.081415,0.062045,0.034671,0.027553,0.020436,0.051357,0.017922,...,0.021652,0.021258,0.060217,0.042963,0.104782,0.087098,0.068396,0.032202,0.041190,0.037369
4,0.027415,0.009719,0.039103,0.019026,0.065426,0.090742,0.019540,0.012399,0.035297,0.042443,...,0.008745,0.006042,0.061651,0.049187,0.008518,0.028883,0.022378,0.060676,0.018439,0.017862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.032074,0.013846,0.054197,0.052935,0.042830,0.046184,0.051953,0.016574,0.034137,0.076278,...,0.036111,0.049952,0.026839,0.049276,0.044627,0.032001,0.041608,0.052652,0.042217,0.033332
5996,0.012516,0.000000,0.051786,0.054696,0.027500,0.039493,0.029016,0.007741,0.032940,0.036727,...,0.015456,0.005350,0.005847,0.034138,0.018550,0.029759,0.023966,0.049398,0.015555,0.023282
5997,0.030333,0.014557,0.029504,0.036809,0.040041,0.040533,0.020146,0.008856,0.039380,0.000000,...,0.009012,0.016210,0.041086,0.182025,0.007460,0.020363,0.017585,0.016345,0.008889,0.013663
5998,0.030527,0.029364,0.054187,0.079735,0.054966,0.071326,0.029804,0.025821,0.038878,0.000000,...,0.010659,0.023349,0.057134,0.217532,0.017189,0.038693,0.032072,0.036336,0.040111,0.004690


In [114]:
dataset.loc[[0, 2, 1577]]

,courseName,universityName,facultyName,isItFullTime,description,startDate,fee,modality,duration,city,country,administration,url,descr_stem,descr_clean,fees_in_usd,descr_len
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,True,3D visualisation and animation play a role i...,September,\nFees\n\nPlease see the university website fo...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , 3d, visualis, and, anim, play, a, role, i...","[3d, visualis, anim, play, role, mani, area, p...",0.0,70
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,True,The Analytical Toxicology MSc is a unique st...,See Course,\nFees\n\nPlease see the university website fo...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , the, analyt, toxicolog, msc, is, a, uniqu...","[the, analyt, toxicolog, msc, uniqu, studi, co...",0.0,27
1577,Counselling and Psychotherapy (Professional ...,University of Salford,School of Health and Society,False,"During this course, you will undertake an in...",September,\nFees\n\nPlease see the university website fo...,"PGDip, MSc",2-3 years Part-time,Manchester,United Kingdom,On Campus,https://www.findamasters.com//masters-degrees/...,"[, , dure, thi, course,, you, will, undertak, ...","[dure, cours, undertak, studi, approach, couns...",0.0,54


### 2nd Inverted Index

Now, let's create the 2nd inverted index ("terms2") dataframe as per the HW description.

We firstly copy our 1st [simple] index into the 2nd one...

In [116]:
terms2 = terms[['term', 'reverse']].copy()

Then, we add another column -- "tfidf_tuples". So now, we can see each term in the "term" column. A list of all the documents that that word has been mentioned in, in the "reverse" column. And finally a tuple of the cosine similarity of each term in each and every document/row it has been mentioned in, in the "tfidf_tuples" column.

In [117]:
terms2['tfidf_tuples'] = terms2.term.apply(lambda term: [(doc_id, tfidf_data.loc[doc_id, term]) 
                                                       for doc_id in tfidf_data.index if tfidf_data.loc[doc_id, term] > 0])

In [118]:
terms2

,term,reverse,tfidf_tuples
0,3d,"[0, 923, 1592, 2033, 2036, 2037, 3239, 3339, 3...","[(0, 0.5692851031776165), (923, 0.158603012853..."
1,visualis,"[0, 78, 306, 323, 470, 471, 474, 738, 754, 796...","[(0, 0.12200966148428337), (78, 0.120143223688..."
2,anim,"[0, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34...","[(0, 0.21799835997509323), (23, 0.499556934030..."
3,play,"[0, 34, 42, 43, 66, 103, 249, 250, 254, 357, 3...","[(0, 0.09450595881405877), (34, 0.073365565236..."
4,role,"[0, 5, 31, 33, 34, 43, 46, 58, 103, 107, 119, ...","[(0, 0.06923786057647283), (5, 0.1026100851575..."
...,...,...,...
8771,recherch,[5994],"[(5994, 0.14270789754346347)]"
8772,mathématiqu,[5994],"[(5994, 0.28541579508692694)]"
8773,allerg,[5995],"[(5995, 0.4069902825964913)]"
8774,slaveri,[5999],"[(5999, 0.15560322326560758)]"


We can see for example [in the above cell's results] that the word "3d" has been mentioned in rows [0, 923, ...]. 

And by looking at the tfidf column we can see that the similarity btwn the word "3d" and doc#0 is 0.5692851031776165.

Furthurmore, the similarity btwn the word "3d" and doc#923 is 0.158603012853.

Etc.

In [119]:
terms2["reverse"]

0       [0, 923, 1592, 2033, 2036, 2037, 3239, 3339, 3...
1       [0, 78, 306, 323, 470, 471, 474, 738, 754, 796...
2       [0, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34...
3       [0, 34, 42, 43, 66, 103, 249, 250, 254, 357, 3...
4       [0, 5, 31, 33, 34, 43, 46, 58, 103, 107, 119, ...
                              ...                        
8771                                               [5994]
8772                                               [5994]
8773                                               [5995]
8774                                               [5999]
8775                                               [5999]
Name: reverse, Length: 8776, dtype: object

In [120]:
terms2["tfidf_tuples"]

0       [(0, 0.5692851031776165), (923, 0.158603012853...
1       [(0, 0.12200966148428337), (78, 0.120143223688...
2       [(0, 0.21799835997509323), (23, 0.499556934030...
3       [(0, 0.09450595881405877), (34, 0.073365565236...
4       [(0, 0.06923786057647283), (5, 0.1026100851575...
                              ...                        
8771                        [(5994, 0.14270789754346347)]
8772                        [(5994, 0.28541579508692694)]
8773                         [(5995, 0.4069902825964913)]
8774                        [(5999, 0.15560322326560758)]
8775                        [(5999, 0.15560322326560758)]
Name: tfidf_tuples, Length: 8776, dtype: object

Again, for easier use, we can save our 2nd inverted index into a csv file...

In [121]:
terms2.to_csv('inverted_index2.csv', index=False)

Now, we are ready to create the function for our 2nd [more advanced search engine]:

In [122]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def execute_query2(query, inverted_index, dataset, tfidf_data):
    # Preprocess the query to match the document processing
    query_terms = [porterstemmer.stem(word) for word in query.lower().split()] 
    
    # Find documents that contain all terms in the query
    relevant_docs = set()
    for term in query_terms:
        if term in inverted_index['term'].values:
            docs_with_term = [doc_id for doc_id, _ in inverted_index.loc[inverted_index['term'] == term, 'tfidf_tuples'].iloc[0]]
            if not relevant_docs:
                relevant_docs = set(docs_with_term)
            else:
                relevant_docs.intersection_update(docs_with_term)
    
    if not relevant_docs:
        return pd.DataFrame()  # If no documents are found, return an empty DataFrame
    
    # Calculate cosine similarity for each document
    query_vec = tfidf.transform([' '.join(query_terms)])
    similarities = {}
    for doc_id in relevant_docs:
        doc_vec = tfidf_data.loc[doc_id].values.reshape(1, -1)
        similarities[doc_id] = cosine_similarity(query_vec, doc_vec)[0][0]
    
    # Sort documents by their similarity score
    sorted_doc_ids = sorted(similarities, key=similarities.get, reverse=True)
    
    # Prepare the results with the similarity score
    results = dataset.loc[sorted_doc_ids, ['courseName', 'universityName', 'description', 'url']]
    results['similarity'] = [similarities[doc_id] for doc_id in sorted_doc_ids]
    
    return results.reset_index(drop=True)

1st Example usage:

In [123]:
# Example usage
results = execute_query2("advanced knowledge", terms2, dataset, tfidf_data)
(results)

,courseName,universityName,description,url,similarity
0,Geo-information Science and Earth Observatio...,University of Twente,INTERESTED IN A CAREER IN SPATIAL DATA SCIEN...,https://www.findamasters.com//masters-degrees/...,0.110091
1,Advanced Mechanical Engineering - MSc/PGDip,Heriot-Watt University,This programme aims to develop the knowledge...,https://www.findamasters.com//masters-degrees/...,0.065608
2,Gerontological Nursing - MSc/PgDip,Trinity College Dublin,The aim of this course is to strengthen and ...,https://www.findamasters.com//masters-degrees/...,0.047973
3,Ageing - MSc,Lancaster University,MSc Ageing aims to allow people working in a...,https://www.findamasters.com//masters-degrees/...,0.045909
4,Environmental Technology - MSc,Imperial College London,The MSc in Environmental Technology provides...,https://www.findamasters.com//masters-degrees/...,0.044955
...,...,...,...,...,...
466,International Business - MSc,University of Glasgow,International Business will provide you with...,https://www.findamasters.com//masters-degrees/...,0.000000
467,Diabetes Practice - MSc,Cardiff University,Why study this courseThe MSc Diabetes Practi...,https://www.findamasters.com//masters-degrees/...,0.000000
468,Diabetes Practice MSc/PGDip/PGCert,Swansea University,Diabetes Practice Course OverviewIf you are ...,https://www.findamasters.com//masters-degrees/...,0.000000
469,Diagnostic Imaging - MSc,Glasgow Caledonian University,Designed to train a new generation of leader...,https://www.findamasters.com//masters-degrees/...,0.000000


2nd Example usage:

In [124]:
# Example usage
results = execute_query2("financial management", terms2, dataset, tfidf_data)
(results)

,courseName,universityName,description,url,similarity
0,Master’s in Finance,Nova School of Business and Economics,The Master’s in Finance prepares students fo...,https://www.findamasters.com//masters-degrees/...,0.160618
1,Banking and Finance MSc,Università Cattolica del Sacro Cuore,The Banking and Finance programme is a speci...,https://www.findamasters.com//masters-degrees/...,0.112317
2,Master Financial Management and Control,The Hague University of Applied Sciences,Reasons for choosing this master programmeYo...,https://www.findamasters.com//masters-degrees/...,0.098304
3,"Finance, Accounting and Management - MSc",University of Bradford,A Master's degree designed to produce fully ...,https://www.findamasters.com//masters-degrees/...,0.094280
4,MSc Finance and Investment Management,University of Liverpool,A career in finance and investment managemen...,https://www.findamasters.com//masters-degrees/...,0.090326
...,...,...,...,...,...
238,Master in Global Logistics and Supply Chain ...,Kuehne Logistics University - KLU,Our MSc in Global Logistics and Supply Chain...,https://www.findamasters.com//masters-degrees/...,0.000000
239,Master of Finance,York University (Canada),The “must-have” degree for specialized finan...,https://www.findamasters.com//masters-degrees/...,0.000000
240,Financial Technology (Fintech) MSc,University of Exeter,OverviewGain an education in finance and eco...,https://www.findamasters.com//masters-degrees/...,0.000000
241,Construction Project Management MSc,University of Portsmouth,OverviewEstablish yourself in the global con...,https://www.findamasters.com//masters-degrees/...,0.000000


The search results seem to be pretty good!

# Q4

Reading the stored CSV files (inverted indexes)

In [ ]:
#terms1 = pd.read_csv("inverted_index.csv")
#terms2 = pd.read_csv("inverted_index2.csv")
